<a href="https://colab.research.google.com/github/chaitsugale/PythonPractice/blob/main/mlflow_and_zenml_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install "zenml[server]"
!zenml integration install sklearn -y
%pip install pyparsing==2.4.2

import IPython

#automatically restart kernel

IPython.Application.instance().kernel.do_shutdown(restart = True)

NumExpr defaulting to 2 threads.
⠴ Installing integrations...
  Using cached pyparsing-2.4.2-py2.py3-none-any.whl (65 kB)
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 2.4.7
    Uninstalling pyparsing-2.4.7:
      Successfully uninstalled pyparsing-2.4.7


{'status': 'ok', 'restart': True}

In [1]:
!rm -rf .zen
!zenml init

NumExpr defaulting to 2 threads.
Initializing the ZenML global configuration version to 0.60.0
⠋ Initializing ZenML repository at /content.
Creating database tables
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
⠏ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
Creating default workspace 'default' ...
⠹ Initializing ZenML repository at /content.
Creating default stack in workspace default...
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML reposi

# Experimental ML Code

In [7]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

def train_test() -> None:
  digits = load_digits()
  data = digits.images.reshape((len(digits.images),-1))
  X_train,X_test,y_train,y_test = train_test_split(
      data,digits.target,test_size=0.2,shuffle = False
  )
  model = SVC(gamma = 0.002)
  model.fit(X_train,y_train)
  test_acc= model.score(X_test,y_test)
  print(f"Test accuracy: {test_acc}")

train_test()


Test accuracy: 0.9527777777777777


#Turning Experiments into ML pipeline with ZenML

In [11]:
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple

@step
def importer() -> Tuple[
  Annotated[np.ndarray,"X_train"],
  Annotated[np.ndarray,"X_test"],
  Annotated[np.ndarray,"Y_train"],
  Annotated[np.ndarray,"y_test"],]:
  digits = load_digits()
  data = digits.images.reshape((len(digits.images),-1))
  X_train,X_test,y_train,y_test = train_test_split(
      data,digits.target,test_size = 0.2,shuffle = False
  )
  return X_train,X_test,y_train,y_test


@step
def svc_trainer(
    X_train : np.ndarray,
    y_train : np.ndarray,) ->ClassifierMixin:
    model = SVC(gamma = 0.001)
    model.fit(X_train,y_train)
    return model

@step
def evaluator(X_test:np.ndarray,
              y_test:np.ndarray,
              model:ClassifierMixin,
              ) -> float:
              test_acc = model.score(X_test,y_test)
              print(f" Test Accuracy : {test_acc} ")
              return test_acc






In [12]:
from zenml import pipeline

@pipeline
def digits_pipeline():
  X_train,X_test,y_train,y_test = importer()
  model = svc_trainer(X_train=X_train,y_train = y_train)
  evaluator(X_test = X_test,y_test = y_test,model = model)


# Running ZenML Pipeline

In [13]:
digits_svc_pipeline = digits_pipeline()

Initiating a new run for the pipeline: digits_pipeline.
Registered new version: (version 2).
Executing a new run.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.
Using cached version of importer.
Step importer has started.
Step svc_trainer has started.
Step svc_trainer has finished in 0.545s.
Step svc_trainer completed successfully.
Step evaluator has started.
 Test Accuracy : 0.9583333333333334 
Step evaluator has finished in 0.580s.
Step evaluator completed successfully.
Pipeline run has finished in 1.837s.
